## Value Investing Stock Screener 



In this project, stocks in the S&P 500 index are screened and given an overall "value" score which takes into account their **Price-to-Earnings**, **Price-to-Book-Value**, and **Enterprise-Value-to-Free-Cash-Flow** ratios. 


The **Finnhub** API is used to retrieve current stock-market data. 

In [1]:
import finnhub
import math
import pandas as pd
import time

### Retrieving Data 
First, the stocks in the S&P 500 are retrieved using the Index's Constituents endpoint (https://finnhub.io/docs/api/indices-constituents).

Then, the ratios of interest are retrieved through the Basic Financials endpoint (https://finnhub.io/docs/api/company-basic-financials).

In [2]:
from api_secrets import FINNHUB_API_KEY

# Setup client
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# Retrieve all the stock tickers in the S&P 500 Index
sp500 = finnhub_client.indices_const(symbol = "^GSPC")['constituents']


In [3]:
# setup dictionary which has the stock symbols as keys, and their ratios as the values (also stored as a dictionary)
ratios = {}

# loop over stocks in index, adding key-value pairs to dict
for symbol in sp500:
    res = finnhub_client.company_basic_financials(symbol, 'metric')
    try:
        pe = res['metric']['peNormalizedAnnual']
        pb = res['metric']['pbAnnual']
        ev_to_cashflow = res['metric']['currentEv/freeCashFlowAnnual']
        ratios[symbol] = {'pe':pe, 'pb':pb, 'ev/cashflow': ev_to_cashflow}
    except KeyError:
        print('missing values detected for: ', symbol)
    
    time.sleep(1) # necessary to deal with the 60-calls-per-minute rate limit that comes with the free api. 
                  # change according to your rate limit
    
# the code above will take at least 500 seconds to run with the free rate limit

missing values detected for:  DISCA


### Wrangling the Data and Creating a DataFrame
Below the dictionary is filtered to remove stocks with missing ratio values. Missing values for the ratios indicate that a stock has negative earnings or cash flows. A value-oriented investor will likely look down upon such stocks, so it is reasonable that they would be removed from a list of value-oriented stock recommendations. 

In [4]:
# needs to use a copy of ratios so that loop indices work properly
filtered_ratios = dict(ratios)

for symbol in ratios.keys():
    if(ratios[symbol]['pb'] == None or ratios[symbol]['pe'] == None or ratios[symbol]['ev/cashflow'] == None):
        del filtered_ratios[symbol]
        
# replace original ratios with filtered ratios
ratios = filtered_ratios


A pandas DataFrame is made to represent the data. Each observation (row) in the DataFrame contains the name of the stock, as well its financial ratios that were obtained earlier. In addition, each ratio value has a corresponding percentile value indicating its percentile relative to the entire column (this will be computed later). Finally, each stock will be given an overall score from 0-100. 

In [9]:
# setup columns of dataframe
value_metrics_dataframe_cols = [
    'symbol',
    'price', 
    'number of shares to buy',
    'pe ratio', 
    'pe percentile',
    'pb ratio', 
    'pb percentile',
    'ev-to-cf ratio',
    'ev-to-cf percentile',
    'overall score (higher is better)'
]

# setup dataframe
value_metrics_dataframe = pd.DataFrame(columns = value_metrics_dataframe_cols)

# add rows incrementally to the end of the dataframe
for symbol in ratios.keys():
    price = finnhub_client.quote(symbol)['c']
    time.sleep(1) # necessary for api rate limit
    value_metrics_dataframe.loc[len(value_metrics_dataframe.index)] = [
        symbol, 
        price,
        'N/A',
        ratios[symbol]['pe'],
        'N/A',
        ratios[symbol]['pb'],
        'N/A',
        ratios[symbol]['ev/cashflow'],
        'N/A',
        'N/A'
    ]
    
n_head = 5 # used to indicate the number of elements to be printed by the dataframe.head() function
value_metrics_dataframe.head(n_head)


,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,EMR,96.06,N/A,18.26727,N/A,5.45179,N/A,28.24030,N/A,N/A
1,TFX,249.63,N/A,27.05342,N/A,3.10520,N/A,22.26750,N/A,N/A
2,SHW,237.33,N/A,33.22317,N/A,26.00187,N/A,48.94488,N/A,N/A
3,GWW,556.25,N/A,28.28209,N/A,15.44533,N/A,45.00158,N/A,N/A
4,WAT,342.58,N/A,30.83780,N/A,57.27585,N/A,32.82070,N/A,N/A


### Percentile and Overall Score Calculation
The percentile of each ratio is computed below, by using `scipy.stats.percentileofscore()`.

In [10]:
from scipy.stats import percentileofscore as score
prefixes = ['pe', 'pb', 'ev-to-cf']

# calculate percentiles for all the ratios, put them in the percentile column
for ratio in prefixes:
    entire_column = value_metrics_dataframe[f'{ratio} ratio'] # entire column for current ratio
    for row in value_metrics_dataframe.index:
        val = value_metrics_dataframe.loc[row, f'{ratio} ratio']
        # set the current percentile column as the percentile of the corresponding ratio value. 
        # division by 100 is to simplify excel formatting 
        value_metrics_dataframe.loc[row, f'{ratio} percentile'] = score(entire_column, val)/100 
        
        
        
value_metrics_dataframe.head(n_head)



,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,EMR,96.06,N/A,18.26727,0.369822,5.45179,0.60355,28.24030,0.39645,N/A
1,TFX,249.63,N/A,27.05342,0.653846,3.10520,0.366864,22.26750,0.301775,N/A
2,SHW,237.33,N/A,33.22317,0.778107,26.00187,0.940828,48.94488,0.713018,N/A
3,GWW,556.25,N/A,28.28209,0.692308,15.44533,0.890533,45.00158,0.671598,N/A
4,WAT,342.58,N/A,30.83780,0.724852,57.27585,0.982249,32.82070,0.494083,N/A


The overall score is defined as $1 - \mu$, where $\mu$ is equal to the mean of the percentiles.

In [11]:
from statistics import mean

for row in value_metrics_dataframe.index:
    percentiles = []
    for ratio in prefixes:
        percentiles.append(value_metrics_dataframe.loc[row, f'{ratio} percentile'])
    
    # mean(percentiles) gives average percentile. higher percentile values indicate a worse value, therefore
    # this mean will be inverted (subtracted from 1) for the overall score calculation
    value_metrics_dataframe.loc[row, 'overall score (higher is better)'] = 1 - mean(percentiles) 

value_metrics_dataframe.head(n_head)


,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,EMR,96.06,N/A,18.26727,0.369822,5.45179,0.60355,28.24030,0.39645,0.543393
1,TFX,249.63,N/A,27.05342,0.653846,3.10520,0.366864,22.26750,0.301775,0.559172
2,SHW,237.33,N/A,33.22317,0.778107,26.00187,0.940828,48.94488,0.713018,0.189349
3,GWW,556.25,N/A,28.28209,0.692308,15.44533,0.890533,45.00158,0.671598,0.248521
4,WAT,342.58,N/A,30.83780,0.724852,57.27585,0.982249,32.82070,0.494083,0.266272


### Selecting the 20 best stocks
Below the data is sorted so that the stocks with a higher overall score are at the top. This would mean that they are undervalued relative to other investments. Then, only the top 20 stocks are selected, by slicing the dataframe. 

In [12]:
value_metrics_dataframe.sort_values('overall score (higher is better)', ascending = False, inplace = True)
value_metrics_dataframe.reset_index(inplace = True, drop = True)

value_metrics_dataframe = value_metrics_dataframe[:20]

value_metrics_dataframe.head(n_head)

,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,PARA,16.88,N/A,4.39260,0.017751,0.48191,0.005917,5.90150,0.011834,0.988166
1,BIO,420.49,N/A,2.90135,0.008876,0.90383,0.035503,2.67873,0.002959,0.984221
2,F,11.63,N/A,2.39102,0.002959,0.96487,0.038462,7.21985,0.029586,0.976331
3,DXC,26.50,N/A,5.01804,0.026627,1.24789,0.06213,5.09027,0.005917,0.968442
4,DISH,14.04,N/A,3.81103,0.014793,0.49263,0.008876,12.42414,0.088757,0.962525


### Calculating the Number of Shares to Buy
The program now asks for the user's input to calculate the size of the positions and the number of shares to buy. 

In [13]:
portfolio_size = input('Enter the size of your portfolio: ')

# loop while float conversion is not successful due to type difference (ValueError)
while(True):
    try:
        portfolio_size = float(portfolio_size)
        break # exit loop, everything successful
    except ValueError:
        portfolio_size = input('Input was not numerical, please try again \nEnter the size of your portfolio: ')
        continue # continue looping until there is a numerical input


Enter the size of your portfolio: 1000000


The size of each position is calculated below. Each stock is given an equal weighting when calculating the number of shares to buy.

In [14]:
position_size = portfolio_size/len(value_metrics_dataframe.index)

for row in value_metrics_dataframe.index:
    # math.floor is used to remove fractional shares
    # the number of stocks to buy = (position size) / (price of the stock)
    value_metrics_dataframe.loc[row, 'number of shares to buy'] =  math.floor(position_size/value_metrics_dataframe.loc[row, 'price'])

In [15]:
value_metrics_dataframe # print the dataframe in full

,symbol,price,number of shares to buy,pe ratio,pe percentile,pb ratio,pb percentile,ev-to-cf ratio,ev-to-cf percentile,overall score (higher is better)
0,PARA,16.88,2962,4.39260,0.017751,0.48191,0.005917,5.90150,0.011834,0.988166
1,BIO,420.49,118,2.90135,0.008876,0.90383,0.035503,2.67873,0.002959,0.984221
2,F,11.63,4299,2.39102,0.002959,0.96487,0.038462,7.21985,0.029586,0.976331
3,DXC,26.50,1886,5.01804,0.026627,1.24789,0.06213,5.09027,0.005917,0.968442
4,DISH,14.04,3561,3.81103,0.014793,0.49263,0.008876,12.42414,0.088757,0.962525
5,MHK,102.22,489,6.43804,0.065089,0.76318,0.017751,8.34117,0.035503,0.960552
6,WDC,31.55,1584,5.23383,0.032544,0.78847,0.023669,11.02672,0.065089,0.959566
7,DHI,89.14,560,5.35581,0.035503,1.58142,0.112426,6.28458,0.017751,0.944773
8,WRK,35.16,1422,5.45009,0.04142,0.79118,0.026627,12.59817,0.097633,0.944773
9,PHM,45.53,1098,5.98934,0.056213,1.52435,0.10355,6.30410,0.02071,0.939842


### Exporting the Data to Excel
The xlsxwriter module is used to export our dataframe to excel.

In [16]:
import xlsxwriter

# setup excel writer and link sheet with dataframe
writer = pd.ExcelWriter('value_screener.xlsx', engine = 'xlsxwriter')
value_metrics_dataframe.to_excel(writer, sheet_name = 'Top 20 Undervalued Stocks', index = False)


Below we will setup the templates necessary for formatting our columns properly.

In [17]:
background_color = '#C4A484'
font_color = '#000000'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

The formats defined above are applied to the appropriate columns, and the names of the columns are set. 

In [18]:
column_formats  = {
    'A': ['Symbol', string_template],
    'B': ['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template],
    'D':['PE ratio', float_template],
    'E':['PE percentile', percent_template],
    'F':['PB ratio', float_template],
    'G':['PB percentile', percent_template],
    'H':['EV/CF ratio', float_template],
    'I':['EV/CF percentile', percent_template],
    'J':['Overall Score (higher is better)', percent_template]
         }

for column in column_formats.keys():
    writer.sheets['Top 20 Undervalued Stocks'].set_column(f'{column}:{column}', 22, column_formats[column][1]) # set format for column
    writer.sheets['Top 20 Undervalued Stocks'].write(f'{column}1', column_formats[column][0], string_template) # set name for column

writer.sheets['Top 20 Undervalued Stocks'].set_column('J:J', 25, column_formats['J'][1]); # overwrite format to fit name


In [19]:
# save the excel output
writer.save()

C:\Users\billy\AppData\Local\Temp\ipykernel_27292\2836777261.py:2: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


### The list of undervalued stocks has now been exported to *value_screener.xlsx* !

**Note**: the workflow for this project was inspired by the *Algorithmic Trading in Python* course (https://github.com/nickmccullum/algorithmic-trading-python). However, there are notable differences in the approach used for obtaining stock-market data (a different API was used for this purpose), and several improvements are made to increase the robustness of the code. For example, a loop was added to continuously check that the user input is numeric when calculating position size. Missing data is also dealt with differently (rows with missing data are removed in this project). 